In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam



print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["label", "sentence"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["label", "sentence"])

train_data = train_df["sentence"].tolist()
test_data = test_df["sentence"].tolist()

train_labels = train_df["label"].tolist()
test_labels = test_df["label"].tolist()

In [ ]:
label_map = {"ham": 0, "spam": 1}
train_labels_encoded = np.array([label_map[label] for label in train_labels])
test_labels_encoded = np.array([label_map[label] for label in test_labels])

In [ ]:
tokenizer = Tokenizer(num_words=1000)  #num_words not necessary
tokenizer.fit_on_texts(train_data)

train_sequences = tokenizer.texts_to_sequences(train_data)
test_sequences = tokenizer.texts_to_sequences(test_data)

max_length = 500

train_padded = pad_sequences(train_sequences, maxlen=max_length)
test_padded = pad_sequences(test_sequences, maxlen=max_length)

print("Train data shape:", train_padded.shape)
print("Test data shape:", test_padded.shape)
print("Train labels shape:", train_labels_encoded.shape)
print("Test labels shape:", test_labels_encoded.shape)
print("Train labels distribution:", np.bincount(train_labels_encoded))
print("Test labels distribution:", np.bincount(test_labels_encoded))

In [ ]:
i = tf.keras.layers.Input(shape=[500])
x = tf.keras.layers.Embedding(10000, 50, input_length=500)(i)
x = tf.keras.layers.LSTM(64)(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.models.Model(inputs=i, outputs=x)

#model = keras.Sequential([
#  keras.layers.Embedding(10000, 64, input_length=100),
 # keras.layers.LSTM(64),
  #keras.layers.Flatten(),
  #keras.layers.Dense(64, activation='relu'),
  #keras.layers.Dense(1, activation='sigmoid')
#])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


In [ ]:
#model.fit(train_padded, train_labels_encoded, epochs=5, batch_size=32, validation_data=(test_padded, test_labels_encoded))
model.fit(train_padded, train_labels_encoded,
             batch_size=128, epochs=10,
              validation_split=0.2,
              callbacks=[tf.keras.callbacks.EarlyStopping(
                  monitor='val_loss', min_delta=0.0001)])

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text = [pred_text]

  tokenizer = Tokenizer(num_words=1000)

  tokenizer.fit_on_texts(pred_text)

  single_sequence = tokenizer.texts_to_sequences(pred_text)
  single_padded = pad_sequences(single_sequence, maxlen=500, padding='post', truncating='post')
  print("Padded",single_padded)

  prediction = model.predict(single_padded)
  if prediction < 0.5:
    predicted_label = 'ham'
  else:
    predicted_label = 'spam'

  final_prediction= [prediction[0][0], predicted_label]
  print(final_prediction)
  return final_prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
